# Overview

This notebook shows you how to put Landsat data into an AWS deployment of the Open Data Cube.

## Credentials

We use the Boto3 library to work with AWS. Saving data to S3 requires credentials with permission to put data into an S3 bucket. Obviously, if you're contributing to this notebook, you shouldn't commit credentials to the repository. 

In [4]:
import boto3

In [12]:
s3 = boto3.resource('s3')
bucket = s3.Bucket('test-odc-bucket')
for obj in bucket.objects.all():
    print(obj.key)

LT05_L1TP_043027_19860805_20161004_01_T1.xml
LT05_L1TP_043027_19860805_20161004_01_T1_ANG.txt
LT05_L1TP_043027_19860805_20161004_01_T1_GCP.txt
LT05_L1TP_043027_19860805_20161004_01_T1_MTL.txt
LT05_L1TP_043027_19860805_20161004_01_T1_VER.jpg
LT05_L1TP_043027_19860805_20161004_01_T1_VER.txt
LT05_L1TP_043027_19860805_20161004_01_T1_bqa.tif
LT05_L1TP_043027_19860805_20161004_01_T1_cfmask.tif
LT05_L1TP_043027_19860805_20161004_01_T1_cfmask_conf.tif
LT05_L1TP_043027_19860805_20161004_01_T1_sr_atmos_opacity.tif
LT05_L1TP_043027_19860805_20161004_01_T1_sr_band1.tif
LT05_L1TP_043027_19860805_20161004_01_T1_sr_band2.tif
LT05_L1TP_043027_19860805_20161004_01_T1_sr_band3.tif
LT05_L1TP_043027_19860805_20161004_01_T1_sr_band4.tif
LT05_L1TP_043027_19860805_20161004_01_T1_sr_band5.tif
LT05_L1TP_043027_19860805_20161004_01_T1_sr_band7.tif
LT05_L1TP_043027_19860805_20161004_01_T1_sr_cloud_qa.tif
LT05_L1TP_043027_19860805_20161004_01_T1_toa_band1.tif
LT05_L1TP_043027_19860805_20161004_01_T1_toa_band2.tif

## 2. Saving a Scene

In [6]:
import requests
import gzip
import tarfile
import io

In [ ]:
def download(url):
    """"""
    r = requests.get(url, stream=True)
    r.raw.decode_content = True
    data = io.BytesIO(r.raw)
    return data

In [9]:
def expand_and_put(path):
    """"""
    with tarfile.open(path) as tar:
        for name in tar.getnames():
            print(name)
            data = tar.extractfile(name).read()
            bucket.put_object(Key=name, Body=data)

In [11]:
url = 'data/foo.tar.gz'
data = expand_and_put(url)

LT05_L1TP_043027_19860720_20161004_01_T1_bqa.tif
LT05_L1TP_043027_19860720_20161004_01_T1_VER.jpg
LT05_L1TP_043027_19860720_20161004_01_T1_toa_band1.tif
LT05_L1TP_043027_19860720_20161004_01_T1_cfmask_conf.tif
LT05_L1TP_043027_19860720_20161004_01_T1_GCP.txt
LT05_L1TP_043027_19860720_20161004_01_T1_sr_band1.tif
LT05_L1TP_043027_19860720_20161004_01_T1_sr_band3.tif
LT05_L1TP_043027_19860720_20161004_01_T1_toa_band7.tif
LT05_L1TP_043027_19860720_20161004_01_T1_sr_band5.tif
LT05_L1TP_043027_19860720_20161004_01_T1_toa_band5.tif
LT05_L1TP_043027_19860720_20161004_01_T1_ANG.txt
LT05_L1TP_043027_19860720_20161004_01_T1_cfmask.tif
LT05_L1TP_043027_19860720_20161004_01_T1_sr_band4.tif
LT05_L1TP_043027_19860720_20161004_01_T1.xml
LT05_L1TP_043027_19860720_20161004_01_T1_VER.txt
LT05_L1TP_043027_19860720_20161004_01_T1_sr_atmos_opacity.tif
LT05_L1TP_043027_19860720_20161004_01_T1_toa_band4.tif
LT05_L1TP_043027_19860720_20161004_01_T1_toa_band2.tif
LT05_L1TP_043027_19860720_20161004_01_T1_toa_ban

## 3. Build a Dataset YAML

## 4. Build an Index